Script to fit a line and an exponential to the meanResponse over blocks. For a visualization of the data-to-be fitted, please see plt_MeanResponseOverBlocks_perCell_perImage.ipynb

Note: for the fit to work, must load manifest

August 2018
Created by M. Sol Bernardez Sarria in collaboration with project teammates and SWDB_2018 TAs


In [ ]:
import matplotlib
%matplotlib notebook

import os
import numpy as np
import pandas as pd

import sys

import matplotlib as mpl
import matplotlib.pyplot as plt

# this makes print statements complaint with python 3 formatting
from __future__ import print_function

# from datetime import datetime

# #Behavior
# from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
# from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis 

# # Local of the event drive path
# event_drive_path_AWS = '/data/dynamic-brain-workshop/visual_behavior_events' #AWS Location
# drive_path =  '/data/dynamic-brain-workshop/visual_behavior'

#from functions developed by/with our group
import oBehave.helperfunctions as obhf
import oBehave.behavior_anaylsis as obba
import oBehave.perCell_functions as obpc

In [ ]:
filename = ('data_manifest.pkl')
data_inv = pd.read_pickle(filename)
data_inv.head()

In [ ]:
#confirm number of expected experiments, compare w/ add_FlashRepeats_Block_BlockMeanResp.ipynb
len(data_inv)

In [ ]:
from scipy.optimize import curve_fit

def fitExp(x,a,b,c):
    return a * np.exp(-b * x) + c

def fitLin(x,a,b):
    return a + b*x 

def r_sq(y, x, f, popt):    
    ss_res = np.sum((y- f(x, *popt))**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

In [ ]:
keylist = ['experiment_id','session_type','cell','image_name',
           'Optim_params_exp', 'r_sq_exp', 'Optim_params_lin', 'r_sq_lin']
fit_dict = {k:[] for k in keylist}

counter = 1
for expID,row in data_inv.iterrows():
    expID = row.experiment_id
    tfilename = ('data_wBlock_mean:', str(expID), '.csv')
    filename = ''.join(tfilename)
    exp_df = pd.read_csv(filename, index_col= 0)
    sessType = row.session_type
    
    for cell in exp_df['cell'].unique():
        for n, im in enumerate(exp_df['image_name'].unique()):
            ty = exp_df[(exp_df.cell.values == cell)&(exp_df.image_name.values == im)&(exp_df.repeats.values == 1)]
            y = ty.three_repeats_mr.values
            x0 = np.arange(0,len(y),1)
            check = np.isnan(y)
            if check.any():
                print('cell', cell, im, 'has nan' )
                continue
            else:
                try:
                    poptE, pcovE = curve_fit(fitExp, x0, y)
                    poptL, pcovL = curve_fit(fitLin, x0, y)
                    R_sq_Exp =r_sq(y, x0, fitExp, poptE)
                    R_sq_Lin =r_sq(y, x0, fitLin, poptL)
                    
                    fit_dict['experiment_id'].append(expID)
                    fit_dict['session_type'].append(sessType)
                    fit_dict['cell'].append(cell)
                    fit_dict['image_name'].append(im),
                    fit_dict['Optim_params_lin'].append([poptL, pcovL])
                    fit_dict['r_sq_lin'].append(R_sq_Lin)            
                    fit_dict['Optim_params_exp'].append([poptE, pcovE])
                    fit_dict['r_sq_exp'].append(R_sq_Exp)

                except RuntimeError:
                    poptE, pcovE = np.nan, np.nan
                    poptL, pcovL = np.nan, np.nan
                    R_sq_Exp = np.nan
                    R_sq_Lin = np.nan
                
                fit_df = pd.DataFrame.from_dict(fit_dict)
                
              
    print('experiment loop ended', counter, ':', row.experiment_id)
    counter = counter +1
                


In [ ]:
#Display fit_df to survey whether it includes data for all experiments
fit_df

In [ ]:
#Save fit_df to not re-calculate it over and over
filename = ('fit_df:ALL.pkl')
fit_df.to_pickle(filename)

filename = ('data_manifest.pkl')
data_inv.to_pickle(filename)